In [1]:
!pip install autogluon
!pip install numpy pandas
!pip install scikit-learn

In [19]:
import pandas as pd
import numpy as np
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

input_data = pd.read_csv("../2023_smartFarm_AI_hackathon_dataset.csv")
input_data.head()

,frmDist,date,inTp,inHd,otmsuplyqy,acSlrdQy,cunt,ph,outTp,outWs,...,WaterCost,FertilizerUsage,FertilizerCost,HeatingEnergyUsage,HeatingEnergyCost,CO2Usage,CO2Cost,MistUsageTime,Mist Cost,HeatingEnergyUsage_cumsum
0,DBSF1059,20220926,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0000,0.000000,0.000000,0.0
1,DBSF1059,20221001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,202.133127,4711.308821,5936.249114,0.0,0.0,0.000000,0.0000,190.168990,2221.173803,0.0
2,DBSF1059,20221002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.484903,271.379890,341.938662,0.0,0.0,0.000000,0.0000,0.000000,0.000000,0.0
3,DBSF1059,20221003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,61.833760,1539.505534,1939.776973,0.0,0.0,253.744916,149.7095,0.000000,0.000000,0.0
4,DBSF1059,20221004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,88.591295,2614.973165,3294.866187,0.0,0.0,3032.885084,1789.4022,1.149739,13.428955,0.0


In [21]:
input_data['date'] = pd.to_datetime(input_data['date'], format='%Y%m%d')
input_data.rename(columns={'date': 'timestamp'}, inplace=True)

input_data['item_id'] = input_data['timestamp'].dt.year.astype(str) + "_" + input_data['timestamp'].dt.month.astype(str)

#input_data = TimeSeriesDataFrame(
#  input_data,
#  id_column="item_id",
#  timestamp_column="timestamp"
#)

input_data.head()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /Users/bello/anaconda3/envs/tomato/lib/python3.9/site-packages/pandas/core/indexes/base.py:3802  │
│ in get_loc                                                                                       │
│                                                                                                  │
│   3799 │   │   │   │   )                                                                         │
│   3800 │   │   │   casted_key = self._maybe_cast_indexer(key)                                    │
│   3801 │   │   │   try:                                                                          │
│ ❱ 3802 │   │   │   │   return self._engine.get_loc(casted_key)                                   │
│   3803 │   │   │   except KeyError as err:                                                       │
│   3804 │   │   │   │   raise KeyError(key) from err                                              │
│   3805 │   │   │   except TypeError:                                                             │
│                                                                                                  │
│ in pandas._libs.index.IndexEngine.get_loc:138                                                    │
│                                                                                                  │
│ in pandas._libs.index.IndexEngine.get_loc:165                                                    │
│                                                                                                  │
│ in pandas._libs.hashtable.PyObjectHashTable.get_item:5745                                        │
│                                                                                                  │
│ in pandas._libs.hashtable.PyObjectHashTable.get_item:5753                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'date'

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱  1 input_data['date'] = pd.to_datetime(input_data['date'], format='%Y%m%d')                    │
│    2 input_data.rename(columns={'date': 'timestamp'}, inplace=True)                              │
│    3                                                                                             │
│    4 input_data['item_id'] = input_data['timestamp'].dt.year.astype(str) + "_" + input_data['    │
│                                                                                                  │
│ /Users/bello/anaconda3/envs/tomato/lib/python3.9/site-packages/pandas/core/frame.py:3807 in      │
│ __getitem__                                                                                      │
│                                                                                                  │
│    3804 │   │   if is_single_key:                                                                │
│    3805 │   │   │   if self.columns.nlevels > 1:                                                 │
│    3806 │   │   │   │   return self._getitem_multilevel(key)                                     │
│ ❱  3807 │   │   │   indexer = self.columns.get_loc(key)                                          │
│    3808 │   │   │   if is_integer(indexer):                                                      │
│    3809 │   │   │   │   indexer = [indexer]                                                      │
│    3810 │   │   else:                                                                            │
│                                                                         

In [9]:
time_limit = 3600 * 1 # hrs
prediction_length = 168 * 1 #weeks
eval_metric = 'MASE'
output_directory = './autogluon_model_log' # output directory
target = 'HeatingEnergyUsage_cumsum'
presets = 'best_quality'

predictor = TimeSeriesPredictor(
    prediction_length=168,
    path=output_directory,
    target=target,
    eval_metric=eval_metric,
)

predictor.fit(
    input_data,
    presets=presets,
    time_limit=time_limit,
)

### result (leaderboard)
leaderboard = predictor.leaderboard(silent=False)
print(leaderboard)

log_path = os.path.join(output_directory, 'train.log')
with open(log_path, 'w') as f:
    f.write(str(leaderboard))

print(f"Logs saved to {log_path}")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /Users/bello/anaconda3/envs/tomato/lib/python3.9/site-packages/autogluon/timeseries/predictor.py │
│ :198 in _check_and_prepare_data_frame                                                            │
│                                                                                                  │
│   195 │   │   if not isinstance(df, TimeSeriesDataFrame):                                        │
│   196 │   │   │   if isinstance(df, pd.DataFrame):                                               │
│   197 │   │   │   │   try:                                                                       │
│ ❱ 198 │   │   │   │   │   df = TimeSeriesDataFrame(df)                                           │
│   199 │   │   │   │   except:                                                                    │
│   200 │   │   │   │   │   raise ValueError(                                                      │
│   201 │   │   │   │   │   │   f"Provided data of type {type(df)} cannot be automatically conve   │
│                                                                                                  │
│ /Users/bello/anaconda3/envs/tomato/lib/python3.9/site-packages/autogluon/timeseries/dataset/ts_d │
│ ataframe.py:109 in __init__                                                                      │
│                                                                                                  │
│   106 │   │   │   if isinstance(data.index, pd.MultiIndex):                                      │
│   107 │   │   │   │   self._validate_multi_index_data_frame(data)                                │
│   108 │   │   │   else:                                                                          │
│ ❱ 109 │   │   │   │   data = self._construct_pandas_frame_from_data_frame(data)                  │
│   110 │   │   elif isinstance(data, str):                                                        │
│   111 │   │   │   data = self._load_data_frame_from_file(data)                                   │
│   112 │   │   elif isinstance(data, Iterable):                                                   │
│                                                                                                  │
│ /Users/bello/anaconda3/envs/tomato/lib/python3.9/site-packages/autogluon/timeseries/dataset/ts_d │
│ ataframe.py:344 in _construct_pandas_frame_from_data_frame                                       │
│                                                                                                  │
│   341 │   │   if TIMESTAMP in df.columns:                                                        │
│   342 │   │   │   df[TIMESTAMP] = pd.to_datetime(df[TIMESTAMP])                                  │
│   343 │   │                                                                                      │
│ ❱ 344 │   │   cls._validate_data_frame(df)                                                       │
│   345 │   │   return df.set_index([ITEMID, TIMESTAMP])                                           │
│   346 │                                                                                          │
│   347 │   @classmethod                                                                           │
│                                                                                                  │
│ /Users/bello/anaconda3/envs/tomato/lib/python3.9/site-packages/autogluon/timeseries/dataset/ts_d │
│ ataframe.py:222 in _validate_data_frame                                                          │
│                                                                                                  │
│   219 │   │   if not isinstance(df, pd.DataFrame):                                               │
│   220 │   │   │   raise ValueError(f"data must be a pd.DataFrame, got {type(df)}")               │
│   221 │   │   if ITEMID not in df.columns:                 

In [ ]:
### data preprocess
# 데이터셋을 data, target으로 변수분리
data = input_data.iloc[:,:-1]
target = input_data.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

In [ ]:
### RMSE, R2 evaluateß
# AutoGluon 모델을 저장
loaded_predictor = TimeSeriesPredictor.load(output_directory)  # 모델의 저장 경로를 지정

# 테스트 데이터 준비
test_df = TimeSeriesDataFrame(X_test) 
prediction = loaded_predictor.predict(test_df)

# RMSE, R2 계산
rmse = np.sqrt(mean_squared_error(y_test, prediction))
r2score = r2_score(y_test, prediction)

In [ ]:
### OUTPUT ###
print("RMSE:", rmse)
print("R2_score:", r2score)